# Comparing SelectKBest to SelectPercentile on a subset of data

Using small batch of data - ## SUBJECT COUNT  HERE

### SelectKBest, k=500

![IMG](images/all_batch0_SVM_nimask.png)  
![IMG](images/all_batch1_SVM_nimask.png)  


### SelectPercentile, percentile=10 (10%)

![IMG](images/percentiles/all_batch0_SVM_nimask.png)  
![IMG](images/percentiles/all_batch1_SVM_nimask.png)  


## Comparison on one subject, all 4 waves

### SelectKBest, k=500 | Nested CV score: 0.6369  
![IMG](images/sub-001_SVM_nimask.png)
### SelectPercentile, 5% | Nested CV score: 0.6116  
![IMG](images/sub-001_SVM_5_nimask.png)

### Batch 0:

In [ ]:
import os
import numpy as np
import nilearn
import glob
import nibabel as nib
import pandas as pd 
from sklearn.model_selection import cross_val_score
from nilearn.input_data import NiftiMasker 
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Agg')



#image mask
imag_mask='/projects/niblab/bids_projects/Experiments/ChocoData/images/bin_mask.nii.gz'


#our behavioral csv file 
stim = '/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/all_batch1.csv'

#our dataset concatenated image 
dataset='/projects/niblab/bids_projects/Experiments/ChocoData/images/all_batch1.nii.gz'
#load behavioral data into a pandas df
behavioral = pd.read_csv(stim, sep="\t")



#grab conditional labels and set up milkshake

behavioral["Label"] = behavioral.replace(['HF_LS_receipt', 'LF_LS_receipt', 'LF_HS_receipt', 'HF_HS_receipt'], 'milkshake')

y = behavioral["Label"]
print(y.unique())


#restrict data to our target analysis 
condition_mask = behavioral["Label"].isin(['milkshake', "h20_receipt"])
y = y[condition_mask]
#confirm we have the # of condtions needed
print(y.unique())

masker = NiftiMasker(mask_img=imag_mask, standardize=True, memory="nilearn_cache", memory_level=1)
X = masker.fit_transform(dataset)
# Apply our condition_mask
X = X[condition_mask]



# PREDICTION FUNCTION
from sklearn.svm import SVC
svc = SVC(kernel='linear')

# FEATURE SELECTION
from sklearn.feature_selection import SelectPercentile, f_classif
feature_selection = SelectPercentile(f_classif, percentile=10)

from sklearn.pipeline import Pipeline

anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])
anova_svc.fit(X,y)
y_pred = anova_svc.predict(X)

# NESTED CROSS VALIDATION 
nested_cv_scores = cross_val_score(anova_svc, X, y,  cv=5) 
#cv_scores = cross_val_score(anova_svc, X, conditions,)
# Print the results
print("Nested CV score: %.4f" % np.mean(nested_cv_scores))

# Here is the image 
coef = svc.coef_
# reverse feature selection
coef = feature_selection.inverse_transform(coef)
# reverse masking
weight_img = masker.inverse_transform(coef)


# Use the mean image as a background to avoid relying on anatomical data
from nilearn import image
mean_img = image.mean_img(dataset)
mean_img.to_filename('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/milkshake_vs_h2O/images/all/all_batch1_mean_nimask.nii')

# Create the figure
from nilearn.plotting import plot_stat_map, show
display = plot_stat_map(weight_img, mean_img, title='Milkshake vs. h2O')
display.savefig('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/milkshake_vs_h2O/images/all/all_batch1_SVM_nimask.png')
# Saving the results as a Nifti file may also be important
weight_img.to_filename('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/milkshake_vs_h2O/images/all/all_batch1_SVM_nimask.nii')





### Batch 1:

In [ ]:
import os
import numpy as np
import nilearn
import glob
import nibabel as nib
import pandas as pd 
from sklearn.model_selection import cross_val_score
from nilearn.input_data import NiftiMasker 
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Agg')



#image mask
imag_mask='/projects/niblab/bids_projects/Experiments/ChocoData/images/bin_mask.nii.gz'


#our behavioral csv file 
stim = '/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/all_batch0.csv'

#our dataset concatenated image 
dataset='/projects/niblab/bids_projects/Experiments/ChocoData/images/all_batch0.nii.gz'
#load behavioral data into a pandas df
behavioral = pd.read_csv(stim, sep="\t")



#grab conditional labels and set up milkshake

behavioral["Label"] = behavioral.replace(['HF_LS_receipt', 'LF_LS_receipt', 'LF_HS_receipt', 'HF_HS_receipt'], 'milkshake')

y = behavioral["Label"]
print(y.unique())




#restrict data to our target analysis 
condition_mask = behavioral["Label"].isin(['milkshake', "h20_receipt"])
y = y[condition_mask]
#confirm we have the # of condtions needed
print(y.unique())


masker = NiftiMasker(mask_img=imag_mask, standardize=True, memory="nilearn_cache", memory_level=1)
X = masker.fit_transform(dataset)
# Apply our condition_mask
X = X[condition_mask]



# PREDICTION FUNCTION
from sklearn.svm import SVC
svc = SVC(kernel='linear')

# FEATURE SELECTION
from sklearn.feature_selection import SelectPercentile, SelectKBest, f_classif
feature_selection = SelectKBest(f_classif, k=500)

from sklearn.pipeline import Pipeline

anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])
anova_svc.fit(X,y)
y_pred = anova_svc.predict(X)

# NESTED CROSS VALIDATION 
nested_cv_scores = cross_val_score(anova_svc, X, y,  cv=5) 
#cv_scores = cross_val_score(anova_svc, X, conditions,)
# Print the results
print("Nested CV score: %.4f" % np.mean(nested_cv_scores))

# Here is the image 
coef = svc.coef_
# reverse feature selection
coef = feature_selection.inverse_transform(coef)
# reverse masking
weight_img = masker.inverse_transform(coef)


# Use the mean image as a background to avoid relying on anatomical data
from nilearn import image
mean_img = image.mean_img(dataset)
mean_img.to_filename('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/milkshake_vs_h2O/images/all_batch0_mean_nimask.nii')

# Create the figure
from nilearn.plotting import plot_stat_map, show
display = plot_stat_map(weight_img, mean_img, title='Milkshake vs. h2O')
display.savefig('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/milkshake_vs_h2O/images/all_batch0_SVM_nimask.png')
# Saving the results as a Nifti file may also be important
weight_img.to_filename('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/milkshake_vs_h2O/images/all_batch0_SVM_nimask.nii')


### One Subject:

In [ ]:
import os
import numpy as np
import nilearn
import glob
import nibabel as nib
import pandas as pd 
from sklearn.model_selection import cross_val_score
from nilearn.input_data import NiftiMasker 
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Agg')



#image mask
imag_mask='/projects/niblab/bids_projects/Experiments/ChocoData/images/bin_mask.nii.gz'


#our behavioral csv file 
stim = '/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/sub-001.csv'

#our dataset concatenated image 
dataset='/projects/niblab/bids_projects/Experiments/ChocoData/images/sub-001.nii.gz'
#load behavioral data into a pandas df
behavioral = pd.read_csv(stim, sep="\t")



#grab conditional labels and set up milkshake

behavioral["Label"] = behavioral.replace(['HF_LS_receipt', 'LF_LS_receipt', 'LF_HS_receipt', 'HF_HS_receipt'], 'milkshake')

y = behavioral["Label"]
print(y.unique())


#restrict data to our target analysis 
condition_mask = behavioral["Label"].isin(['milkshake', "h20_receipt"])
y = y[condition_mask]
#confirm we have the # of condtions needed
print(y.unique())


masker = NiftiMasker(mask_img=imag_mask, standardize=True, memory="nilearn_cache", memory_level=1)
X = masker.fit_transform(dataset)
# Apply our condition_mask
X = X[condition_mask]



# PREDICTION FUNCTION
from sklearn.svm import SVC
svc = SVC(kernel='linear')

# FEATURE SELECTION
from sklearn.feature_selection import SelectPercentile, f_classif, SelectKBest
#feature_selection = SelectPercentile(f_classif, percentile=5)
feature_selection = SelectKBest(f_classif, k=500)

from sklearn.pipeline import Pipeline

anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])
anova_svc.fit(X,y)
y_pred = anova_svc.predict(X)

# NESTED CROSS VALIDATION 
nested_cv_scores = cross_val_score(anova_svc, X, y,  cv=5) 
#cv_scores = cross_val_score(anova_svc, X, conditions,)
# Print the results
print("Nested CV score: %.4f" % np.mean(nested_cv_scores))

# Here is the image 
coef = svc.coef_
# reverse feature selection
coef = feature_selection.inverse_transform(coef)
# reverse masking
weight_img = masker.inverse_transform(coef)


# Use the mean image as a background to avoid relying on anatomical data
from nilearn import image
mean_img = image.mean_img(dataset)
mean_img.to_filename('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/milkshake_vs_h2O/images/4w_sub-001_mean_nimask.nii')

# Create the figure
from nilearn.plotting import plot_stat_map, show
display = plot_stat_map(weight_img, mean_img, title='Milkshake vs. h2O , 4 waves')
display.savefig('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/milkshake_vs_h2O/images/4w_sub-001_SVM_nimask.png')
# Saving the results as a Nifti file may also be important
# sub-001_SVM_5_nimask.nii
weight_img.to_filename('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/milkshake_vs_h2O/images/4w_sub-001_SVM_nimask.nii')

